<a href="https://colab.research.google.com/github/chauhan231405/bigdataLAB/blob/main/Spark1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=a854cdeb5133b5e1f1c742af795651f3cb427a7cbdf27d24d13eb39f17508c56
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
from pyspark import SparkConf,SparkContext
conf=SparkConf().setAppName('abc').setMaster('local') #
sc=SparkContext(conf=conf)
sc.setLogLevel('ERROR')
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('abc').config('','').getOrCreate()
# NUMPY Dense Vector
import numpy as np
v1=np.array([1,2,3,4,5])
print(v1)
# simple python list
v2=[1,2,3,4,5,6]
print(v2)
# Sparce & dense spark vector
from pyspark.mllib.linalg import Vectors
v3=Vectors.dense([3,4,5,6])
print(v3)
v4 = Vectors.sparse(3, [0, 2], [1.0, 3.0])
print(v4)

[1 2 3 4 5]
[1, 2, 3, 4, 5, 6]
[3.0,4.0,5.0,6.0]
(3,[0,2],[1.0,3.0])


In [3]:
df = spark.read.csv('data.csv',header=True,inferSchema=True) # header = None
df.show(5,0) # 0 doesnot truncate displaying columns, useful in large dataset

AnalysisException: ignored

In [ ]:
# Step 3: Exploratory Data Analysis

In [4]:
df.count()

NameError: ignored

In [ ]:
df.select('Grades').distinct().count()

14

In [ ]:
df.printSchema()

root
 |-- Time_to_Study: integer (nullable = true)
 |-- Grades: double (nullable = true)



In [ ]:
df.show()

+-------------+------+
|Time_to_Study|Grades|
+-------------+------+
|            1|   1.5|
|            5|   2.7|
|            7|   3.1|
|            3|   2.1|
|            2|   1.8|
|            9|   3.9|
|            6|   2.9|
|           12|   4.5|
|           11|   4.3|
|            2|   1.8|
|            4|   2.4|
|            8|   3.5|
|           13|   4.8|
|            9|   3.9|
|           14|   5.0|
|           10|   4.1|
|            6|   2.9|
|           12|   4.5|
|            1|   1.5|
|            4|   2.4|
+-------------+------+
only showing top 20 rows



In [ ]:
df.describe().show()

+-------+-----------------+------------------+
|summary|    Time_to_Study|            Grades|
+-------+-----------------+------------------+
|  count|               50|                50|
|   mean|             7.12|3.2220000000000004|
| stddev|4.048884956102742|1.1047744252164082|
|    min|                1|               1.5|
|    max|               14|               5.0|
+-------+-----------------+------------------+



In [ ]:
#create a feature array by omitting the last column
feature_cols = df.columns[:-1]
from pyspark.ml.feature import VectorAssembler
vect_assembler = VectorAssembler(inputCols = feature_cols, outputCol="features")
#Utilize Assembler created above in order to add the feature column
data_w_features = vect_assembler.transform(df)

In [ ]:
finalized_data = data_w_features.select("features","Grades")
finalized_data.show()

+--------+------+
|features|Grades|
+--------+------+
|   [1.0]|   1.5|
|   [5.0]|   2.7|
|   [7.0]|   3.1|
|   [3.0]|   2.1|
|   [2.0]|   1.8|
|   [9.0]|   3.9|
|   [6.0]|   2.9|
|  [12.0]|   4.5|
|  [11.0]|   4.3|
|   [2.0]|   1.8|
|   [4.0]|   2.4|
|   [8.0]|   3.5|
|  [13.0]|   4.8|
|   [9.0]|   3.9|
|  [14.0]|   5.0|
|  [10.0]|   4.1|
|   [6.0]|   2.9|
|  [12.0]|   4.5|
|   [1.0]|   1.5|
|   [4.0]|   2.4|
+--------+------+
only showing top 20 rows



In [ ]:
# train test split
train_dataset, test_dataset = finalized_data.randomSplit([0.7, 0.3])
print(df.count())
print(train_dataset.count())
print(test_dataset.count())

50
42
8


In [ ]:
#Import Linear Regression class called LinearRegression
from pyspark.ml.regression import LinearRegression
LinReg = LinearRegression(featuresCol="features", labelCol="Grades")

In [ ]:
# Model training and testing
#Train the model on the training using fit() method.
model = LinReg.fit(train_dataset)
#Predict the Grades using the evulate method
pred = model.evaluate(test_dataset)

In [ ]:
pred.predictions.show()

+--------+------+------------------+
|features|Grades|        prediction|
+--------+------+------------------+
|   [1.0]|   1.5| 1.559778872831174|
|   [1.0]|   1.5| 1.559778872831174|
|   [2.0]|   1.8|1.8325686363824594|
|   [2.0]|   1.8|1.8325686363824594|
|   [5.0]|   2.7|2.6509379270363165|
|  [12.0]|   4.5| 4.560466271895316|
|  [12.0]|   4.5| 4.560466271895316|
|  [14.0]|   5.0| 5.106045798997887|
+--------+------+------------------+



In [ ]:
#Find out coefficient value
coefficient = model.coefficients
print ("The coefficient of the model is : %a" %coefficient)
#Find out intercept Value
intercept = model.intercept
print ("The Intercept of the model is : %f" %intercept)

The coefficient of the model is : DenseVector([0.2728])
The Intercept of the model is : 1.286989


In [ ]:
#Evaluate the model using metric like Mean Absolute Error(MAE), Root Mean␣↪Square Error(RMSE) and R-Square
from pyspark.ml.evaluation import RegressionEvaluator
evaluation = RegressionEvaluator(labelCol="Grades", predictionCol="prediction")
# Root Mean Square Error
rmse = evaluation.evaluate(pred.predictions, {evaluation.metricName: "rmse"})
print("RMSE: %.3f" % rmse)
# Mean Square Error
mse = evaluation.evaluate(pred.predictions, {evaluation.metricName: "mse"})
print("MSE: %.3f" % mse)
# Mean Absolute Error
mae = evaluation.evaluate(pred.predictions, {evaluation.metricName: "mae"})
print("MAE: %.3f" % mae)
# r2 - coefficient of determination
r2 = evaluation.evaluate(pred.predictions, {evaluation.metricName: "r2"})
print("r2: %.3f" %r2)

RMSE: 0.061
MSE: 0.004
MAE: 0.058
r2: 0.998
